In [172]:
# Python
import pandas as pd
import numpy as np
import torch
from requests.packages import target
#
from xgboost import XGBRegressor
#
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [173]:
df = pd.read_csv('../../../data/df_monthly_returns_complete_percentage.csv', index_col='Date')

df_tabular = pd.read_csv('../../../data/train_test/df_tabular.csv', index_col=0)

X_train = pd.read_csv('../../../data/train_test/df_tabular_xtrain.csv', index_col=0)
y_train = pd.read_csv('../../../data/train_test/df_tabular_ytrain.csv', index_col=0)
X_test = pd.read_csv('../../../data/train_test/df_tabular_xtest.csv', index_col=0)
y_test = pd.read_csv('../../../data/train_test/df_tabular_ytest.csv', index_col=0)

In [189]:
df.head()

,RS1.L,KE,TEG.DE,LEG.DE,SCS,HNI,AVT,ACCO,VNA.DE,7912.T,...,DEQ.DE,KIDS,HALO,MATW,9842.T,KVHI,MOON.L,NEO,6055.T,UNP
Date,,,,,,,,,,,,,,,,,,,,,
1999-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-11-01,1.13,1.00,1.00,1.00,1.03,1.10,1.01,1.03,1.00,1.03,...,1.00,1.00,1.00,0.93,1.00,0.99,1.00,1.00,1.00,0.84
1999-12-01,1.11,1.00,1.00,1.00,0.92,1.02,1.10,1.00,1.00,0.97,...,1.00,1.00,1.00,1.18,1.00,1.01,1.00,1.00,1.00,0.93
2000-01-01,1.02,1.00,1.00,1.00,0.96,0.88,0.89,0.73,1.00,1.00,...,1.00,1.00,1.00,0.85,1.00,1.22,1.00,1.00,1.00,0.93
2000-02-01,1.00,1.01,0.81,1.23,1.08,0.96,1.25,1.95,1.18,0.93,...,1.03,0.94,1.07,1.04,1.47,1.97,1.13,0.95,1.01,0.94


In [175]:
df.mean(axis=1)

Date
1999-10-01         NaN
1999-11-01    1.025547
1999-12-01    1.033835
2000-01-01    0.992874
2000-02-01    5.930569
                ...   
2024-05-01    1.034162
2024-06-01    0.993243
2024-07-01    1.055251
2024-08-01    0.997199
2024-09-01    0.966679
Length: 300, dtype: float64

In [190]:
X_train.head()

,month,year,m_return(t-12),m_return(t-11),m_return(t-10),m_return(t-9),m_return(t-8),m_return(t-7),m_return(t-6),m_return(t-5),m_return(t-4),m_return(t-3),m_return(t-2),m_return(t-1),m_return(t),ticker_label
2,12,1999,1.00,1.00,1.00,1.00,1.0,1.01,1.00,1.00,1.00,1.00,1.0,1.02,1.11,1371
3,1,2000,1.01,1.00,1.00,1.00,1.0,1.00,1.01,1.00,1.00,1.00,1.0,1.00,1.02,1371
4,2,2000,1.00,1.01,1.00,1.00,1.0,1.00,1.00,1.01,1.00,1.00,1.0,1.00,1.00,1371
5,3,2000,1.00,1.00,1.01,1.00,1.0,1.00,1.00,1.00,1.01,1.00,1.0,1.00,1.00,1371
6,4,2000,1.00,1.00,1.00,1.01,1.0,1.00,1.00,1.00,1.00,1.01,1.0,1.00,1.00,1371


In [191]:
y_test.head()

,m_return_target(t+1)
255,1.09
256,1.01
257,1.11
258,1.02
259,1.07


In [178]:
reg = xgboost.XGBRegressor(n_estimators=1000)
reg.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        #early_stopping_rounds=50,
        verbose=False) # Change verbose to True if you want to see it train

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [179]:
# Predictions
y_train_pred = reg.predict(X_train)
y_test_pred = reg.predict(X_test)

In [180]:
# True values
y_true = df.mean(axis=1)

In [182]:
y_true

Date
1999-10-01         NaN
1999-11-01    1.025547
1999-12-01    1.033835
2000-01-01    0.992874
2000-02-01    5.930569
                ...   
2024-05-01    1.034162
2024-06-01    0.993243
2024-07-01    1.055251
2024-08-01    0.997199
2024-09-01    0.966679
Length: 300, dtype: float64

In [185]:

train_pred_torch_list = torch.from_numpy(y_train_pred)
# Reshape to (num_samples, num_features) for normalization
train_rows = int(len(train_pred_torch_list)/1653)
train_pred_torch_view = train_pred_torch_list.view(train_rows, 1653)
df_train_pred_view = pd.DataFrame(train_pred_torch_view)
y_train_true = df_train_pred_view.mean(axis=1)
y_train_true

0      1.008120
1      1.013285
2      1.010209
3      1.012696
4      1.011994
         ...   
248    1.018307
249    1.013816
250    1.014403
251    1.010942
252    1.015797
Length: 253, dtype: float32

In [186]:
test_pred_torch_list = torch.from_numpy(y_test_pred)
test_rows = int(len(test_pred_torch_list) / 1653)

test_pred_torch_view = test_pred_torch_list.view(test_rows, 1653)
df_test_pred_view = pd.DataFrame(test_pred_torch_view)
y_test_true = df_test_pred_view.mean(axis=1)
y_test_true

0     1.079504
1     1.128966
2     1.150882
3     1.105346
4     1.138556
5     1.098432
6     1.144861
7     1.079334
8     1.143317
9     1.132314
10    1.156715
11    1.112353
12    1.100364
13    1.093954
14    1.098792
15    1.130769
16    1.074374
17    1.152937
18    1.134711
19    1.141034
20    1.145708
21    1.187630
22    1.139091
23    1.140404
24    1.095342
25    1.135841
26    1.152137
27    1.137754
28    1.192320
29    1.126222
30    1.128906
31    1.106057
32    1.204554
33    1.143878
34    1.146310
35    1.126790
36    1.109124
37    1.080688
38    1.104218
39    1.114362
40    1.091730
41    1.188110
42    1.125517
43    1.107383
44    1.132453
dtype: float32

In [187]:
#min_date = pd.to_datetime(df_tabular.index[-1]).replace(day=1) - pd.DateOffset(months=60)
#min_datestr = min_date.strftime('%Y-%m-%d')
min_datestr = '2020-01-0.1'

In [188]:

# Create the plot
fig = go.Figure()

# Add the timeseries line
fig.add_trace(go.Scatter(y=y_true, x=df.index.tolist(), mode='lines', name='Actual returns',
                         line=dict(color='#5c839f', width=2)))  #, line=dict(color='red'))
# Add the training plot in red
fig.add_trace(go.Scatter(y=y_train_true, x=df.index.tolist()[:len(y_train_true)],
                         mode='lines', name='Train returns',
                         line=dict(color='red', width=2)))  #, line=dict(color='red')

# Add the testing plot in green
fig.add_trace(go.Scatter(y=y_test_true, x=df.index.tolist()[len(y_train_true):],
                         mode='lines', name='Test returns',
                         line=dict(color='green', width=2)))  # , line=dict(color='green')

fig.add_vline(x=min_datestr, line_color='red', line_dash='dash', line_width=1)

# Update layout with labels
fig.update_layout(
        title='1 Year Prediction vs Actual Plot',
        xaxis=dict(
                title='Date'
        ),
        yaxis=dict(
                title='Day closing return (%)',
                tickformat='.0%',
                range=[0.75, 1.6]
        ),
        legend=dict(title="Legend"),
        template="plotly_white"
)

fig.show()